In [52]:
import chess
import RL_utils
from stockfish import Stockfish

In [50]:
# Our existing CNN model
model_path = "models/TORCH_250EPOCH 1.pth"

## Load stockfish

In [ ]:
from RL_utils import PositionEvaluator

In [ ]:
stockfish_path = "models/stockfish/stockfish-windows-x86-64-avx2.exe"
evaluator = PositionEvaluator(stockfish_path, elo_rating=1400)
evaluator.get_parameters()

{'Debug Log File': '',
 'Contempt': 0,
 'Min Split Depth': 0,
 'Ponder': 'false',
 'MultiPV': 1,
 'Skill Level': 20,
 'Move Overhead': 10,
 'Minimum Thinking Time': 20,
 'Slow Mover': 100,
 'UCI_Chess960': 'false',
 'UCI_LimitStrength': True,
 'UCI_Elo': 1200,
 'Threads': 1,
 'Hash': 16}

In [48]:
board = chess.Board()
board.push_uci("e2e4")
board.push_uci("h7h6")

Move.from_uci('h7h6')

In [ ]:
evaluate_position(board)

0.83

In [49]:
engine.set_fen_position(board.fen())
engine.get_top_moves(3)

[{'Move': 'd2d4', 'Centipawn': 83, 'Mate': None},
 {'Move': 'g1f3', 'Centipawn': 71, 'Mate': None},
 {'Move': 'b1c3', 'Centipawn': 68, 'Mate': None}]

## Load our CNN trained model

In [55]:
model, device = RL_utils.load_model(model_path)
model = model.to(device)

In [57]:
import torch
dummy_input = torch.randn(1, 19, 8, 8).to(device)
with torch.no_grad():
    output = model(dummy_input)
    print(f"Model output shape: {output.shape}")  # Should be [1, 4288]
    print("Model loaded successfully!")

Model output shape: torch.Size([1, 4288])
Model loaded successfully!


In [ ]:
engine.quit()